In [1]:
import torch
import os
from torch.utils.data import TensorDataset, DataLoader
from models import *
from utils import *
import os, sys, time, datetime, random
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable
import cv2
from PIL import Image

In [3]:
config_path='config/yolov3.cfg'
weights_path='config/yolov3.weights'
class_path='config/coco.names'
img_size=416
conf_thres=0.5
nms_thres=0.4


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# load model and put into eval mode
model = Darknet(config_path, img_size=img_size)
model.load_weights(weights_path)
model.to(device)
model.eval()
data_transforms = transforms.Compose([ transforms.ToTensor()])
if device == torch.device("cpu"):
    Tensor = torch.FloatTensor
else:
    Tensor = torch.cuda.FloatTensor
    
    
    
f_rnn = torch.load('f_rnn128_20e.pt')                                           #Forward  LSTM
b_rnn = torch.load('b_best.pt')                                                 #Backward LSTM
cnn_model = torch.load('ensembledModel.jfc_6,200_75e.pt',map_location = 'cpu')  #Ensambled CNN
if train_on_gpu:
    f_rnn.cuda()
    b_rnn.cuda()
    cnn_model.cuda()

In [ ]:

frames_tensor = torch.FloatTensor(1,3,256,256).to(device)
cropped_frames_tensor = torch.FloatTensor(1,3,256,256).to(device)

sequence_length = 100
batch_size = 256

video = ' '    
cap = cv2.VideoCapture(video)
mot_tracker = Sort()
frames = 0
box_h = 556
box_w = 556
y1 = 300
x1 = 300
idx=0
n = 0


In [ ]:
def detect_image(img):
    
    # scale and pad image
    ratio = min(img_size/img.size[0], img_size/img.size[1])
    imw = round(img.size[0] * ratio)
    imh = round(img.size[1] * ratio)
    img_transforms = transforms.Compose([ transforms.Resize((imh, imw)),

         transforms.Pad((max(int((imh-imw)/2),0), max(int((imw-imh)/2),0), max(int((imh-imw)/2),0), max(int((imw-imh)/2),0)),
                        (128,128,128)),
         transforms.ToTensor(),
         ])
    
    
    # convert image to Tensor
    image_tensor = img_transforms(img).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input_img = Variable(image_tensor.type(Tensor))
    
    
    # run inference on the model and get detections
    with torch.no_grad():
        detections = model(input_img)
        detections = utils.non_max_suppression(detections, 80, conf_thres, nms_thres)
    return detections[0]


#######################################################################################################################

def chunk(features,seq_length):
    for i in range(0, len(features)-seq_length, 1):
        yield features[i:i+seq_length]
             
########################################################################################################################
def batching_data(cnn_out, sequence_length, batch_size, shuffling):
    
    rNpArr = np.flip(cnn_out.cpu().numpy(),0).copy()   
    reversed_cnn_out = torch.from_numpy(rNpArr)
    
    feature_tensors = torch.stack(list(chunk(cnn_out, sequence_length)))
    reversed_feature_tensors = torch.stack(list(chunk(reversed_cnn_out, sequence_length)))
    
    data = TensorDataset(feature_tensors)
    data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=shuffling,drop_last=False) 
    reversed_data = TensorDataset(reversed_feature_tensors)
    reversed_data_loader = torch.utils.data.DataLoader(reversed_data, batch_size=batch_size,
                                                       shuffle=shuffling,drop_last=False)
    return data_loader, reversed_data_loader



In [ ]:
def inferOnVideo(path_to_video):
    #frames_list = []
    #cropped_frames_list = []
    
    #Reserving a space for frames and cropped frames tensors in GPU
    frames_tensor = torch.FloatTensor(1,3,256,256).to(device)
    cropped_frames_tensor = torch.FloatTensor(1,3,256,256).to(device)
    
    video = ' '
    cap = cv2.VideoCapture(path_to_video)
    mot_tracker = Sort()
    frames = 0
    box_h = 556
    box_w = 556
    y1 = 300
    x1 = 300
    idx=0
    n = 0
    while(True):
        idx+=1
        ret, frame = cap.read()
        if not ret:
            break

            
        frame = cv2.cvtColor( frame, cv2.COLOR_BGR2RGB)
        pilimg = Image.fromarray(frame)        
        detections = detect_image(pilimg)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        img = np.array(pilimg)
        
        pad_x = max(img.shape[0] - img.shape[1], 0) * (img_size / max(img.shape))
        pad_y = max(img.shape[1] - img.shape[0], 0) * (img_size / max(img.shape))
        unpad_h = img_size - pad_y
        unpad_w = img_size - pad_x

        if detections is not None:
            tracked_objects = mot_tracker.update(detections.cpu())
            n_cls_preds = len(unique_labels)

            for x1, y1, x2, y2, obj_id, cls_pred in tracked_objects:
                box_h = int(((y2 - y1) / unpad_h) * img.shape[0])
                box_w = int(((x2 - x1) / unpad_w) * img.shape[1])
                y1 = int(((y1 - pad_y // 2) / unpad_h) * img.shape[0])
                x1 = int(((x1 - pad_x // 2) / unpad_w) * img.shape[1])
                
            starty = y1-20 if y1>20 else 0    
            startx = x1-20 if x1>20 else 0
            
            roi = frame[starty:y1 + box_h+50, startx:x1 + box_w+50,:]            
        else:
            starty = y1-20 if y1>20 else 0    
            startx = x1-20 if x1>20 else 0
            roi = frame[starty:y1 + box_h+50, startx:x1 + box_w+50,:]
        if idx%2==0:
            n+=1
            frame = data_transforms(cv2.resize(frame, (256, 256))).to(device)
            roi = data_transforms(cv2.resize(roi, (256, 256))).to(device)
            frames_tensor = torch.cat((frames_tensor,frame.unsqueeze(0)),0)
            cropped_frames_tensor = torch.cat((cropped_frames_tensor,roi.unsqueeze(0)),0)
            print(n)
            
    ############################################################################################################################
    frames_tensor = frames_tensor[1:]
    cropped_frames_tensor = cropped_frames_tensor[1:]
    
    data = TensorDataset(frames_tensor, cropped_frames_tensor)
    data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=False,drop_last=False)
    
    ######################################## Extracting Image Features and Predictions using resnet  ################################################
    cnn_out200 = torch.FloatTensor(1,200).cuda()
    cnn_out6 = torch.FloatTensor(1,6).cuda()
    cnn_model.eval()
    k=0
    with torch.no_grad():
        for data,c_data in data_loader:
            k+=1
            print(k)

            if train_on_gpu:
                data  , c_data  = data.cuda(), c_data.cuda()

            out6,out200 = cnn_model.forward(data,c_data)
            cnn_out6 = torch.cat((cnn_out6,out6),0)
            cnn_out200 = torch.cat((cnn_out200,out200),0)
    cnn_out6    = cnn_out6[1:]
    cnn_out200  = cnn_out200[1:]
    del data  , c_data
    ###########################################################################################################################
    
    rnn_batch_size = len(cnn_out200)
    f_data_loader200, b_data_loader200 = batching_data(cnn_out200,sequence_length,batch_size = rnn_batch_size,shuffling = False)
    
    ############################################### Inference #################################################################
    
    f_rnn.eval()
    b_rnn.eval()
    final_out = torch.zeros([len(cnn_out200),6 ], dtype=torch.float64).cuda()
    fout=torch.FloatTensor(1,6).cuda()
    bout=torch.FloatTensor(1,6).cuda()

    with torch.no_grad():
        hiddenf = f_rnn.init_hidden(len(f_data_loader200.dataset))
        hiddenb = b_rnn.init_hidden(len(f_data_loader200.dataset))
        for (fdata,rdata) in zip(f_data_loader200, b_data_loader200):

            if train_on_gpu:
                fdata, rdata = fdata[0].cuda(), rdata[0].cuda()

            outputf, hiddenf = f_rnn(fdata, hiddenf)
            outputb, hiddenb = b_rnn(rdata, hiddenb)

            outputf = F.log_softmax(outputf, dim=1)
            outputb = F.log_softmax(outputb, dim=1)

            fout = torch.cat((fout,outputf),0)
            bout = torch.cat((bout,outputb),0)


    fout = fout[1:]
    bout = bout[1:]   
    final_out[sequence_length:] = fout
    rNpArr = np.flip(bout.cpu().numpy(),0).copy()   
    bout = torch.from_numpy(rNpArr)
    final_out[:-sequence_length] += bout.double().cuda()
    #cnn_tout6 = F.log_softmax(cnn_tout6, dim=1)
    #final_out += cnn_tout6.double()
    final_out /= 2
    _, pred = torch.max(final_out, 1)
    #acc=torch.stack(list(pred==all_ttargets.long()))
    #print('accuracy  : ',(torch.sum(acc).item()/len(acc))*100, '%')
    
    
    ###########################################################################################################################
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    return result

50


In [2]:
from ActionRecognitionAPI import *

In [3]:
out = inferOnVideo('/home/eng_mohammed_saad_18/dataset/test/27_1_crop.mp4')

4063


In [4]:
len(out)

4063

In [5]:
out[0]

0.0

In [8]:
out[400:450]

array([4., 4., 4., 4., 2., 2., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
      dtype=float32)